In [1]:
import random
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import torch,torchvision
from torch.nn import *
from tqdm import tqdm
import cv2
from torch.optim import *
# Preproccessing
from sklearn.preprocessing import (
    StandardScaler,
    RobustScaler,
    MinMaxScaler,
    MaxAbsScaler,
    OneHotEncoder,
    Normalizer,
    Binarizer
)
# Decomposition
from sklearn.decomposition import PCA
from sklearn.decomposition import KernelPCA
# Feature Selection
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import RFECV
from sklearn.feature_selection import SelectFromModel
# Model Eval
from sklearn.compose import make_column_transformer
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score,train_test_split
from sklearn.metrics import mean_absolute_error,mean_squared_error
# Other
import pickle
import wandb

PROJECT_NAME = 'Apple-Stock-Price'
device = 'cuda:0'
np.random.seed(21)
random.seed(21)
torch.manual_seed(21)

In [2]:
data = pd.read_csv('./data.csv')

In [3]:
data

,Date,Open,High,Low,Close,Adj Close,Volume
0,1980-12-12,0.128348,0.128906,0.128348,0.128348,0.100600,469033600
1,1980-12-15,0.122210,0.122210,0.121652,0.121652,0.095352,175884800
2,1980-12-16,0.113281,0.113281,0.112723,0.112723,0.088353,105728000
3,1980-12-17,0.115513,0.116071,0.115513,0.115513,0.090540,86441600
4,1980-12-18,0.118862,0.119420,0.118862,0.118862,0.093165,73449600
...,...,...,...,...,...,...,...
10278,2021-09-17,148.820007,148.820007,145.759995,146.059998,146.059998,129728700
10279,2021-09-20,143.800003,144.839996,141.270004,142.940002,142.940002,123478900
10280,2021-09-21,143.929993,144.600006,142.779999,143.429993,143.429993,75834000
10281,2021-09-22,144.449997,146.429993,143.699997,145.850006,145.850006,76404300


In [4]:
data = data['Open']

In [5]:
data = data.tolist()

In [6]:
data = torch.from_numpy(np.array(data)).view(1,-1).to(device).to(device).float()

In [7]:
data_input = data[:1,:-1].float()
data_target = data[:1,1:].float()

In [8]:
class Model(Module):
    def __init__(self):
        super().__init__()
        self.hidden = 64
        self.lstm1 = LSTMCell(1,self.hidden)
        self.lstm2 = LSTMCell(self.hidden,self.hidden)
        self.lstm3 = LSTMCell(self.hidden,self.hidden)
        self.linear = Linear(self.hidden,1)
    
    def forward(self,X,future=0):
        preds = []
        batch_size = X.size(0)
        h_t1,c_t1 = torch.zeros(batch_size,self.hidden).to(device),torch.zeros(batch_size,self.hidden).to(device)
        h_t2,c_t2 = torch.zeros(batch_size,self.hidden).to(device),torch.zeros(batch_size,self.hidden).to(device)
        h_t3,c_t3 = torch.zeros(batch_size,self.hidden).to(device),torch.zeros(batch_size,self.hidden).to(device)
        for X_batch in X.split(1,dim=1):
            X_batch = X_batch.to(device)
            h_t1,c_t1 = self.lstm1(X_batch,(h_t1,c_t1))
            h_t1,c_t1 = h_t1.to(device),c_t1.to(device)
            h_t2,c_t2 = self.lstm2(h_t1,(h_t2,c_t2))
            h_t2,c_t2 = h_t2.to(device),c_t2.to(device)
            h_t3,c_t3 = self.lstm3(h_t2,(h_t3,c_t3))
            h_t3,c_t3 = h_t3.to(device),c_t3.to(device)
            pred = self.linear(h_t3)
            preds.append(pred)
        for _ in range(future):
            X_batch = X_batch.to(device)
            h_t1,c_t1 = self.lstm1(X_batch,(h_t1,c_t1))
            h_t1,c_t1 = h_t1.to(device),c_t1.to(device)
            h_t2,c_t2 = self.lstm2(h_t1,(h_t2,c_t2))
            h_t2,c_t2 = h_t2.to(device),c_t2.to(device)
            h_t3,c_t3 = self.lstm3(h_t2,(h_t3,c_t3))
            h_t3,c_t3 = h_t3.to(device),c_t3.to(device)
            pred = self.linear(h_t3)
            preds.append(pred)
        preds = torch.cat(preds,dim=1)
        return preds

In [9]:
model = Model().to(device)
criterion = MSELoss()

In [10]:
optimizer = Adam(model.parameters(),lr=0.001)

In [11]:
epochs = 100

In [17]:
# wandb.init(project=PROJECT_NAME,name='baseline')
# for _ in tqdm(range(epochs)):
#     def closure():
#         optimizer.zero_grad()
#         preds = model(data_input)
#         loss = criterion(preds,data_target)
#         loss.backward()
#         wandb.log({'Loss':loss.item()})
#         return loss
#     optimizer.step(closure)
#     with torch.no_grad():
#         n = data_input.shape[1]
#         future = 100
#         preds = model(data_input,future)
#         loss = criterion(preds[:,:-future],data_target)
#         wandb.log({'Val Loss':loss.item()})
#         preds = preds[0].view(-1).cpu().detach().numpy()
#         plt.plot(figsize=(12,6))
#         plt.plot(np.arange(n),data_target.view(-1).cpu().detach().numpy())
#         plt.plot(np.arange(n,n+future),preds[n:])
#         plt.savefig('./img.png')
#         plt.close()
#         wandb.log({'Img':wandb.Image(cv2.imread('./img.png'))})
# wandb.finish()

Exception in thread Thread-17:
Traceback (most recent call last):
  File "/home/indika/anaconda3/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/home/indika/anaconda3/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/home/indika/anaconda3/lib/python3.7/site-packages/wandb/sdk/wandb_run.py", line 203, in check_status
    status_response = self._interface.communicate_stop_status()
  File "/home/indika/anaconda3/lib/python3.7/site-packages/wandb/sdk/interface/interface.py", line 737, in communicate_stop_status
    resp = self._communicate(req, timeout=timeout, local=True)
  File "/home/indika/anaconda3/lib/python3.7/site-packages/wandb/sdk/interface/interface.py", line 539, in _communicate
    return self._communicate_async(rec, local=local).get(timeout=timeout)
  File "/home/indika/anaconda3/lib/python3.7/site-packages/wandb/sdk/interface/interface.py", line 544, in _communicate_async
    raise Exceptio

In [14]:
torch.save(data,'./data.pt')
torch.save(data,'./data.pth')
torch.save(data_input,'data_input.pt')
torch.save(data_input,'data_input.pth')
torch.save(data_target,'data_target.pt')
torch.save(data_target,'data_target.pth')

In [15]:
torch.save(model,'custom-model.pt')
torch.save(model,'custom-model.pth')
torch.save(model.state_dict(),'custom-model-sd.pt')
torch.save(model.state_dict(),'custom-model-sd.pth')
torch.save(model,'model.pt')
torch.save(model,'model.pth')
torch.save(model.state_dict(),'model-sd.pt')
torch.save(model.state_dict(),'model-sd.pth')

In [16]:
torch.save(model,'custom-model.pt')
torch.save(model,'custom-model.pth')
torch.save(model.state_dict(),'custom-model-sd.pt')
torch.save(model.state_dict(),'custom-model-sd.pth')
torch.save(model,'model.pt')
torch.save(model,'model.pth')
torch.save(model.state_dict(),'model-sd.pt')
torch.save(model.state_dict(),'model-sd.pth')